In [4]:
import pandas as pd
import nltk
from tqdm import tqdm
from nltk import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()
#from Levenshtein import distance
import psycopg2
import random
import json
from string import punctuation


# eng

In [2]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''
    
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

# TUR

In [7]:
from TurkishStemmer import TurkishStemmer
stemmer = TurkishStemmer()
import requests

['мама_NOUN_мама_10_1_1', 'мыла_NOUN_мыло_10_1_1']

In [8]:
def get_word_pos_lemma_id_list_tur(clean_ngramm,word_id, ref_id, set_id):
    headers_tr = {
    'cache-control': 'no-cache',
    'content-type': 'application/json',
    'postman-token': 'c18af364-c1cb-cc41-0903-063547ac7fce',
    }
    url_tr = 'http://localhost:8081/analyze'
    
    final_ngramm = []
    tkn = nltk.word_tokenize(clean_ngramm)
    #print(tkn)
    for word in tkn:
        data = {"tokens" : [word]}
        data_dump = json.dumps(data)
        r = requests.post(url = url_tr, data=data_dump, headers=headers_tr)
        lemma = stemmer.stem(word)
        pos = r.text.split("+")[1].lower()
        element = word + "_" + pos + "_" + lemma + '_' + str(word_id) + '_' + str(ref_id) + '_' + str(set_id)
        final_ngramm.append(element)
    return final_ngramm
get_word_pos_lemma_id_list_tur("merhaba", 1,1,1,)

['merhaba_noun_merhap_1_1_1']

# PORT

In [2]:
import spacy
nlp = spacy.load("/Users/lilyakhoang/input/pt_core_news_sm-2.1.0/pt_core_news_sm/pt_core_news_sm-2.1.0")

In [7]:
def get_word_pos_lemma_id_list_port(clean_ngramm,word_id, ref_id, set_id):
    
    final_ngramm = []
    tkn = nltk.word_tokenize(clean_ngramm)
    #print(tkn)
    for word in tkn:
        for token in nlp(word):
            pos = token.pos_
            lemma = token.lemma_
            break
        element = word + "_" + pos + "_" + lemma + '_' + str(word_id) + '_' + str(ref_id) + '_' + str(set_id)
        final_ngramm.append(element)
    return final_ngramm
get_word_pos_lemma_id_list_port("ferara", 1,1,1,)

['ferara_VERB_ferara_1_1_1']

# RUS

In [ ]:
def get_word_pos_lemma_id_list_rus(clean_ngramm,word_id, ref_id, set_id):
    final_ngramm = []
    tkn = nltk.word_tokenize(clean_ngramm)
    #print(tkn)
    for word in tkn:
        p = morph.parse(word)[0]
        lemma = p.normal_form
        pos = p.tag.POS
        #print(lemma,pos)
        if not pos:
            pos = "nodetected"
        element = word + "_" + pos + "_" + lemma + '_' + str(word_id) + '_' + str(ref_id) + '_' + str(set_id)
        final_ngramm.append(element)
    return final_ngramm
get_word_pos_lemma_id_list("мама мыла", 10,1,1)

In [8]:
def  get_lemmatized_popular_ngramm(popular_ngramm, popular_ngramm_ids,popular_ngramm_ref_ids,popular_ngramm_set_ids, debug = False):
    ngramm_list = []
    for word_ind in tqdm(range(len(popular_ngramm))):
        word_id = popular_ngramm_ids[word_ind]
        ngramm = popular_ngramm[word_ind]
        ref_id = popular_ngramm_ref_ids[word_ind]
        set_id = popular_ngramm_set_ids[word_ind]
        clean_ngramm = ''
        for char in ngramm:
            if char not in punctuation: #punctuation_small_set
                clean_ngramm += char
            else:
                clean_ngramm += ' '
        if debug: print(clean_ngramm)
        final_ngramm = get_word_pos_lemma_id_list_port(clean_ngramm,word_id,ref_id,set_id)
        ngramm_list.append(final_ngramm)

    return ngramm_list

In [9]:
def get_alph_dict(ngramms_pos_lemm_list):
    alph_dict = {}
    for ngramm in ngramms_pos_lemm_list:
        for elemetn in ngramm:
            word = elemetn.split("_")[0]
            first_two_el = min(2,len(word))
            first_two_letters = word[:first_two_el]
            if first_two_letters in alph_dict:
                alph_dict[first_two_letters].append(ngramm)
            else:
                alph_dict[first_two_letters] = []
                alph_dict[first_two_letters].append(ngramm)
    return alph_dict

In [10]:
def get_top_prop(db):
    popular_qgramms = pd.read_csv(db)
    popular_qgramms_list = list(popular_qgramms['translate'])
    ppopular_qgramms_list_id = list(popular_qgramms['word_id'])
    popular_qgramms_ref_id = list(popular_qgramms['ref_id'])
    popular_qgramms_set_id = list(popular_qgramms['setting_id'])
    print(popular_qgramms.head())
    return popular_qgramms_list, ppopular_qgramms_list_id, popular_qgramms_ref_id, popular_qgramms_set_id

In [11]:
def save_reference(prefix, word_list, id_list, ref_id_list, set_id_list):
    ngramm = get_lemmatized_popular_ngramm(word_list,id_list,ref_id_list,set_id_list)
    ngramm_dict = get_alph_dict(ngramm)
    with open(prefix +"gramms_top.json" ,"w") as f:
        json.dump(ngramm, f, indent = 4, ensure_ascii=False)
    with open(prefix + "gramms_alpha_dict.json" ,"w") as f:
        json.dump(ngramm_dict, f, indent = 4, ensure_ascii=False)

In [12]:
def get_db_return_ref_files(prefix, db):
    word_list, id_list, ref_id_list, set_id_list= get_top_prop(db)
    save_reference(prefix, word_list, id_list, ref_id_list, set_id_list)
    

In [13]:
get_db_return_ref_files("bi", "top_10k_bigr.csv")

  0%|          | 4/4198 [00:00<01:54, 36.58it/s]

   word_id  ref_id  setting_id                translate
0    77689       3           1            guarda-costas
1   474461       3           1     professor particular
2   474466       3           1  representante comercial
3   474468       3           1     químico farmacêutico
4   474509       3           1         educação musical


100%|██████████| 4198/4198 [02:04<00:00, 33.62it/s]


In [14]:
get_db_return_ref_files("tri", "top_10k_trigramm.csv")

  0%|          | 2/5783 [00:00<05:07, 18.79it/s]

   word_id  ref_id  setting_id                     translate
0     4434       3           1                   dono de bar
1   474448       3           1            fiscal de impostos
2   474458       3           1  trabalhador sem qualificação
3   474464       3           1             agente de seguros
4    38974       3           1           comissária de bordo


100%|██████████| 5783/5783 [03:34<00:00, 26.99it/s]


In [15]:
get_db_return_ref_files("q", "top_10k_qgramm.csv")

  0%|          | 2/2008 [00:00<01:44, 19.18it/s]

   word_id  ref_id  setting_id                   translate
0   296473       3           1  livro de frases essenciais
1   728231       3           1      viajar para o exterior
2   475869       3           1        salto de pára-quedas
3  8816453       3           1      Estar no caminho certo
4  4756594       3           1            Estar por um fio


100%|██████████| 2008/2008 [01:33<00:00, 21.40it/s]
